In [1302]:
with open('23b.txt') as f:
    lines = f.readlines()

In [1303]:
from typing import List, Tuple

In [1304]:
import sys
sys.setrecursionlimit(2000)

### Part 1 and Part 2

Alright so I think my current gameplan is to take the board state, and for each pod, take every position in the board.
Then filter: if the end position is occupied, in front of doorway, is in wrong home, or there is a foreigner in the home that it is trying to go into, remove that position from the list.
if there is another pod in the way, remove it from the list.
if the pod is in the hallway, and the end position is also in the hallway, remove it from the list.
also probably remove it from the list if the end position is the upper home pos and the bottom home pos is free.

after you remove all invalid moves, make a new board state and increment its energy accordingly. Recursively check I guess? if no moves are possible or the end goal is achieved, stop. gotta keep track of the energies. after all possibilities have been checked, find the one with lowest energy.


In [1305]:
MAX_ENERGY = 60000
MAX_MOVES = 40

In [1306]:
points_dict = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [1307]:
# FOR PART 1: USE BELOW SPOTS LISTS
# A_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,3),(3,3)]
# B_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,5),(3,5)]
# C_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,7),(3,7)]
# D_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,9),(3,9)]

# FOR PART 2: USE BELOW SPOTS LISTS
A_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,3),(3,3),(4,3),(5,3)]
B_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,5),(3,5),(4,5),(5,5)]
C_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,7),(3,7),(4,7),(5,7)]
D_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,9),(3,9),(4,9),(5,9)]

In [1308]:
# FOR PART 1: USE THE BELOW base_grid() FUNC.
# def base_grid():
#     grid = [['.']*13 for i in range(5)]
#     grid[0] = ['#'] * 13
#     grid[1][0] = '#'
#     grid[1][12] = '#'
#     grid[2][0:3] = '#' * 3
#     grid[2][10:13] = '#' * 3
#     grid[2][4] = '#'
#     grid[2][6] = '#'
#     grid[2][8] = '#'
#     grid[3][2] = '#'
#     grid[3][4] = '#'
#     grid[3][6] = '#'
#     grid[3][8] = '#'
#     grid[3][10] = '#'
#     grid[4][2:11] = '#' * 9
#     return grid

# FOR PART 2: USE THE BELOW base_grid() FUNC.
def base_grid():
    grid = [['.']*13 for i in range(7)]
    grid[0] = ['#'] * 13
    grid[1][0] = '#'
    grid[1][12] = '#'
    grid[2][0:3] = '#' * 3
    grid[2][10:13] = '#' * 3
    grid[2][4] = '#'
    grid[2][6] = '#'
    grid[2][8] = '#'
    grid[3][2] = '#'
    grid[3][4] = '#'
    grid[3][6] = '#'
    grid[3][8] = '#'
    grid[3][10] = '#'
    grid[4][2] = '#'
    grid[4][4] = '#'
    grid[4][6] = '#'
    grid[4][8] = '#'
    grid[4][10] = '#'
    grid[5][2] = '#'
    grid[5][4] = '#'
    grid[5][6] = '#'
    grid[5][8] = '#'
    grid[5][10] = '#'
    grid[6][2:11] = '#' * 9
    return grid

In [1309]:
class Game_State:
    
    def __init__(self, pods: List[Tuple[int,int,chr]]):
        self.pods = pods
        self.base = base_grid()
        for coords in self.pods:
            row,col,ch = coords
            self.base[row][col] = ch
            
    def check_point(self, row: int, col: int) -> bool:
        if self.base[row][col] != '.':
            return False
        return True

    def check_home(self, flavour: chr) -> bool:
        if flavour == 'A':
            if self.base[2][3] == 'C' or self.base[2][3] == 'B' or self.base[2][3] == 'D': return False
            if self.base[3][3] == 'C' or self.base[3][3] == 'B' or self.base[3][3] == 'D': return False
            # Comment below 2 lines out for part 1. Do the same for other flavours.
            if self.base[4][3] == 'C' or self.base[4][3] == 'B' or self.base[4][3] == 'D': return False
            if self.base[5][3] == 'C' or self.base[5][3] == 'B' or self.base[5][3] == 'D': return False 
        if flavour == 'B':
            if self.base[2][5] == 'A' or self.base[2][5] == 'C' or self.base[2][5] == 'D': return False
            if self.base[3][5] == 'A' or self.base[3][5] == 'C' or self.base[3][5] == 'D': return False
            if self.base[4][5] == 'A' or self.base[4][5] == 'C' or self.base[4][5] == 'D': return False
            if self.base[5][5] == 'A' or self.base[5][5] == 'C' or self.base[5][5] == 'D': return False  
        if flavour == 'C':
            if self.base[2][7] == 'A' or self.base[2][7] == 'B' or self.base[2][7] == 'D': return False
            if self.base[3][7] == 'A' or self.base[3][7] == 'B' or self.base[3][7] == 'D': return False
            if self.base[4][7] == 'A' or self.base[4][7] == 'B' or self.base[4][7] == 'D': return False
            if self.base[5][7] == 'A' or self.base[5][7] == 'B' or self.base[5][7] == 'D': return False  
        if flavour == 'D':
            if self.base[2][9] == 'A' or self.base[2][9] == 'B' or self.base[2][9] == 'C': return False
            if self.base[3][9] == 'A' or self.base[3][9] == 'B' or self.base[3][9] == 'C': return False
            if self.base[4][9] == 'A' or self.base[4][9] == 'B' or self.base[4][9] == 'C': return False
            if self.base[5][9] == 'A' or self.base[5][9] == 'B' or self.base[5][9] == 'C': return False  
        return True
         
    def complete_home(self, col: int) -> bool:
        if col == 3:
            correct = 'A'
        elif col == 5:
            correct = 'B'
        elif col == 7:
            correct = 'C'
        else:
            correct = 'D'
        # below line for part 1.
        # if self.base[2][col] == correct and self.base[3][col] == correct: return True
        # below 2 lines for part 2.
        if self.base[2][col] == correct and self.base[3][col] == correct and \
            self.base[4][col] == correct and self.base[5][col] == correct: return True 
        return False
        
    def complete_total(self) -> bool:
         if self.complete_home(3) and self.complete_home(5) and self.complete_home(7) and self.complete_home(9): return True
         return False
         
    def __str__(self):
        string = ''
        for i in range(len(self.base)):
            string += str(self.base[i])
            if i != len(self.base) - 1: string += '\n'
        return string

In [1310]:
init_pods = []
for row in range(len(lines)):
    for col in range(len(lines[row])):
        if lines[row][col] == 'A':
            init_pods.append((row,col,'A'))
        elif lines[row][col] == 'B':
            init_pods.append((row,col,'B'))
        elif lines[row][col] == 'C':
            init_pods.append((row,col,'C'))
        elif lines[row][col] == 'D':
            init_pods.append((row,col,'D'))
init_state = Game_State(init_pods)

In [1311]:
def get_spots(flavour: chr) -> List[Tuple[int, int]]:
    if flavour == 'A': return A_spots
    elif flavour == 'B': return B_spots
    elif flavour == 'C': return C_spots
    elif flavour == 'D': return D_spots
    else:
        print("error")
        return A_spots

In [1312]:
def correct_spot(column: int, flavour: chr) -> bool:
    if column == 3 and flavour == 'A': return True
    if column == 5 and flavour == 'B': return True
    if column == 7 and flavour == 'C': return True
    if column == 9 and flavour == 'D': return True
    return False

In [1313]:
def valid_end(goal: Tuple[int, int], flavour: chr, hallway: bool, grid: Game_State) -> bool:
        #check if room doesnt contain any foreigners
        row = goal[0]
        col = goal[1]
        if row > 1 and not grid.check_home(flavour):
            return False
        #check that if the bottom home slot if open, the upper one is invalid. right?
        if row > 1 and row < len(lines) - 2 and grid.base[row+1][col] == '.':
            return False
        # print('valid end')
        return True

In [1314]:
def valid_start(start: Tuple[int, int], flav: chr, grid: Game_State) -> bool:
    if correct_spot(start[1], flav):
        if start[0] == 5: return False
        if correct_spot(start[1], grid.base[5][start[1]]):
            if start[0] == 4: return False
            if correct_spot(start[1], grid.base[4][start[1]]):
                if start[0] == 3: return False
                if correct_spot(start[1], grid.base[3][start[1]]):
                    return False
    # print("valid start")
    return True

In [1315]:
def valid_path(start: Tuple[int, int], goal: Tuple[int, int], grid: Game_State) -> bool:
    if start[0] == 1 and goal[0] == 1: return False
    curr_row = start[0]
    curr_col = start[1]
    if curr_row > 1 and goal[0] > 1:
        for i in range(1, curr_row):
            if not grid.check_point(curr_row-i, curr_col): return False
        curr_row = 1
        # check moving right
        if goal[1] > start[1]:
            for i in range(1, goal[1] - start[1] + 1):
                if not grid.check_point(curr_row, start[1] + i):
                    return False
            curr_col = goal[1]
        #check moving left
        if goal[1] < start[1]:
            for i in range(1, start[1] - goal[1] + 1):
                if not grid.check_point(curr_row, start[1] - i):
                    return False
            curr_col = goal[1]
        for i in range(1, goal[0]):
            if not grid.check_point(curr_row+i, curr_col): return False
        return True
    # check if have to move up first
    if goal[0] < start[0]:
        for i in range(1, start[0] - goal[0] + 1):
            if not grid.check_point(start[0] - i, start[1]):
                return False
        curr_row = goal[0]

    # check moving right
    if goal[1] > start[1]:
        for i in range(1, goal[1] - start[1] + 1):
            if not grid.check_point(curr_row, start[1] + i):
                return False
        curr_col = goal[1]
    #check moving left
    if goal[1] < start[1]:
        for i in range(1, start[1] - goal[1] + 1):
            if not grid.check_point(curr_row, start[1] - i):
                return False
        curr_col = goal[1]
    
    #check moving down after
    if goal[0] > start[0]:
        for i in range(1, goal[0] - start[0] + 1):
            if not grid.check_point(start[0] + i, curr_col):
                return False
        curr_row = goal[0]
    # print('valid path')
    return True

        

In [1316]:
def min_dist_to_goal(row: int, col: int, flavour: chr) -> int:
    # use line below for part 1
    # dist = abs(row - 2)
    # use line below for part 2
    dist = abs(row - 3)
    if flavour == 'A':
        dist += abs(col - 3)
    elif flavour == 'B':
        dist += abs(col - 5)
    elif flavour == 'C':
        dist += abs(col - 7)
    elif flavour == 'D':
        dist += abs(col - 9)
    return dist    

In [1317]:
def min_energy_forecast(grid: Game_State) -> int:
    cum = 0
    for pod in grid.pods:
        cum += min_dist_to_goal(pod[0], pod[1], pod[2])
    return cum

In [1318]:
def energy_calc(start: Tuple[int, int], goal: Tuple[int, int], flavour: chr) -> int:
    energy_per = points_dict[flavour]
    if start[0] > 1 and goal[0] > 1:
        num_moves = start[0] - 1 + abs(start[1] - goal[1]) + goal[0] - 1
    else:
        num_moves = abs(goal[0] - start[0]) + abs(goal[1] - start[1])
    return energy_per * num_moves

In [1319]:
def check_complete(grid: Game_State) -> bool:
    if grid.complete_home(3) and grid.complete_home(5) and grid.complete_home(7) and grid.complete_home(9): return True
    return False

In [1320]:
def stepper(grid: Game_State, curr_energy: int, curr_moves: int) -> List[int]:
    global MAX_ENERGY
    if curr_energy > MAX_ENERGY: return []
    if curr_moves > MAX_MOVES: return []
    if grid.complete_total(): 
        # print(curr_energy)
        if curr_energy < MAX_ENERGY:
            MAX_ENERGY = curr_energy
        return [curr_energy]
    if min_energy_forecast(grid) > MAX_ENERGY: return []
    succs = []
    pods = grid.pods
    #preliminary go home check
    for i in range(len(pods)):
        pod = pods[i]
        start = (pod[0], pod[1])
        flav = pod[2]
        all_moves = get_spots(flav)
        valid_moves = []
        for move in all_moves:
            if valid_start(start, flav, grid) and valid_end(move, flav, True if start[0] == 1 else False, grid) and valid_path(start, move, grid):
                # print(move)
                valid_moves.append(move)
                if move[0] > 1:
                    energy_cost = energy_calc(start, move, flav)
                    next_pods = pods.copy()
                    next_pods[i] = (move[0], move[1], flav)
                    next_grid = Game_State(next_pods)
                    next_energy = curr_energy + energy_cost
                    # print(next_grid)
                    # print('\n')
                    succs.extend(stepper(next_grid, next_energy, curr_moves + 1))
                    return succs
    for i in range(len(pods)):
        pod = pods[i]
        start = (pod[0], pod[1])
        flav = pod[2]
        all_moves = get_spots(flav)
        valid_moves = []
        for move in all_moves:
            if valid_start(start, flav, grid) and valid_end(move, flav, True if start[0] == 1 else False, grid) and valid_path(start, move, grid):
                # print(move)
                valid_moves.append(move)

        # print('\n')
        # print(flav)
        # print(grid)
        # print(valid_moves)
        for v_move in valid_moves:
            energy_cost = energy_calc(start, v_move, flav)
            next_pods = pods.copy()
            next_pods[i] = (v_move[0],  v_move[1], flav)
            next_grid = Game_State(next_pods)
            next_energy = curr_energy + energy_cost
            # print(next_grid)
            # print('\n')
            succs.extend(stepper(next_grid, next_energy, curr_moves + 1))
    # print('BREAK')
    return succs


In [1321]:
output = stepper(init_state, 0, 0)
min_energy = MAX_ENERGY
for i in range(len(output)):
    if output[i] < min_energy:
        min_energy = output[i]
print(min_energy)

42308
